In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

In [2]:
# dataset_path = "/home/ns/robosuite/collects/1705258813_3304067/can_jan14_image_166.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/sojib_square_expert/NutAssemblySquare_02_01_2024_06_38PM_sojib/demo_image.hdf5"
f_org = h5py.File(dataset_path, "r")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  40
Max length:  574
Min length:  365
Mean length:  460.775


In [4]:
demos[:3]

['demo-True_1', 'demo-True_10', 'demo-True_11']

In [5]:
len(demos)

40

In [6]:
demos2copy=[demo.decode('utf-8') for demo in f_org['mask']['expert']] 
len(demos2copy)

31

In [7]:
demos2copy

['demo-True_1',
 'demo-True_10',
 'demo-True_12',
 'demo-True_13',
 'demo-True_14',
 'demo-True_15',
 'demo-True_16',
 'demo-True_17',
 'demo-True_18',
 'demo-True_2',
 'demo-True_20',
 'demo-True_21',
 'demo-True_23',
 'demo-True_24',
 'demo-True_25',
 'demo-True_26',
 'demo-True_27',
 'demo-True_28',
 'demo-True_29',
 'demo-True_30',
 'demo-True_31',
 'demo-True_33',
 'demo-True_34',
 'demo-True_35',
 'demo-True_36',
 'demo-True_39',
 'demo-True_4',
 'demo-True_40',
 'demo-True_6',
 'demo-True_7',
 'demo-True_8']

In [5]:
# fn='gd.txt'
# with open(fn, 'r') as f:
#     file_demos = f.readlines()
# demos = [x.strip() for x in file_demos]
# len(demos), demos[:5]

(100, ['demo_1', 'demo_100', 'demo_101', 'demo_105', 'demo_107'])

In [8]:
# dataset_path_sub = "/home/ns/robosuite/collects/1705258813_3304067/can_jan14_image_166_sub.hdf5"
dataset_path_sub = "/home/ns/collect_robomimic_demos/sojib_square_expert/NutAssemblySquare_02_01_2024_06_38PM_sojib/demo_image_sub.hdf5"
f_sub = h5py.File(dataset_path_sub, "w")

In [9]:
keys_to_copy=demos2copy
len(keys_to_copy)

31

In [10]:
f_sub.create_group("data")
f_sub.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [11]:
attrs=f_org["data"].attrs["env_args"]
f_sub["data"].attrs["env_args"] = attrs

In [12]:
i=1
for demo_name in tqdm.tqdm(keys_to_copy):
    b_demo=f_org['data'][demo_name]
    num_samples=b_demo.attrs['num_samples']
    model_file=b_demo.attrs['model_file']

    bdemo_name="demo_"+str(i)
    i = i+1

    f_sub["data"].create_group(bdemo_name)


    ks=['actions', 'dones', 'rewards', 'states']
    for k in ks:
        kdata = b_demo[k]
        f_sub["data"][bdemo_name].create_dataset(k, data=kdata)

    obs_keys =b_demo['obs'].keys()
    next_obs_keys=b_demo['next_obs'].keys()
    for k in obs_keys:
        kdata = b_demo['obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)
    for k in next_obs_keys:
        kdata = b_demo['next_obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'next_obs/{k}', data=kdata)

    f_sub["data"][bdemo_name].attrs["num_samples"] = num_samples
    f_sub["data"][bdemo_name].attrs["model_file"] = model_file
 
f_sub.close()
f_org.close()

  0%|          | 0/31 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:02<00:00, 14.19it/s]


In [13]:
f_sub = h5py.File(dataset_path_sub, "r")
f_sub.keys()

<KeysViewHDF5 ['data', 'mask']>

In [14]:
f_sub['data'].keys()

<KeysViewHDF5 ['demo_1', 'demo_10', 'demo_11', 'demo_12', 'demo_13', 'demo_14', 'demo_15', 'demo_16', 'demo_17', 'demo_18', 'demo_19', 'demo_2', 'demo_20', 'demo_21', 'demo_22', 'demo_23', 'demo_24', 'demo_25', 'demo_26', 'demo_27', 'demo_28', 'demo_29', 'demo_3', 'demo_30', 'demo_31', 'demo_4', 'demo_5', 'demo_6', 'demo_7', 'demo_8', 'demo_9']>

In [15]:
f_sub.close()
f_org.close()